In [1]:
%%time
reload_data = False

import joblib, time, os, copy, datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch.nn import *
from torch.nn.functional import *

import torch
import torchvision

#enable it when running in google cloud, to upload/download file from/to VM to/from google cloud bucket
from google.cloud import storage

bucket_root_path = "weicheng30417"
project_data_folder = "data/breakfast-img-data/"

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#turn on cpu for troubleshooting as gpu doesnt throw proper error message, runing in cpu shows more specific error message
#device = torch.device("cpu") 
print(device)
torch.manual_seed(0)

cuda:0
CPU times: user 356 ms, sys: 40 ms, total: 396 ms
Wall time: 397 ms


In [2]:
def download_all_data_from_bucket():
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    file_id = project_data_folder
    blobs=bucket.list_blobs(prefix=file_id)
    for blob in blobs:        
        file_names = blob.name.split("/")
        if len(file_names) == 3:
            if file_names[2] != "":
                file_name = file_names[2]            
                blob.download_to_filename(file_name)
                print("Download from {0} to local {1}".format(blob.name, file_name))
                

def download_file_from_bucket(file):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    butcketFile = project_data_folder + file
    blob = bucket.blob(butcketFile)
    blob.download_to_filename(file)
    print("Download from {0} to local {1}".format(butcketFile, file))
    
                
def upload_files(files):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    for file in files:
        butcketFile = project_data_folder + file
        blob = bucket.blob(butcketFile)
        blob.upload_from_filename(file)
        print("Upload from local {0} to {1}".format(file, butcketFile))

def upload_all_files():    
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_root_path)
    
    directory = os.getcwd()
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv") or filename.endswith(".ipynb") or filename.endswith(".model"):             
            butcketFile = project_data_folder + file
            blob = bucket.blob(butcketFile)
            blob.upload_from_filename(filename)
            print("Upload from local {0} to {1}".format(filename, butcketFile))        
        
        
def flatten(ls):
     for item in ls:
            for child in list(item):
                yield child

def get_segment_positions(x):
    ps = x.segment.split()
    ls = []
    for i in range(len(ps)-1):
        if i == 0:
            ls.append([int(ps[i]), int(ps[i+1])])
        else:
            ls.append([int(ps[i])+1, int(ps[i+1])])        
    return ls

def get_segment_features(x):
    ls = []
    for rg in x.positions:
        ls.append(x.feature[rg[0]:rg[1]])            
    return ls

def splitDataFrameList(df,target_column):
    def splitListToRows(row,row_accumulator,target_column):
        split_row = row[target_column]
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column))
    new_df = pd.DataFrame(new_rows)
    return new_df
                
def get_train_data():
    training_segment = pd.read_csv('training_segment.txt', header=None, names = ['segment'])
    training_segment['feature'] = joblib.load('train_feature.joblib')
    training_segment['positions'] = training_segment.apply(lambda x: get_segment_positions(x), axis=1)
    training_segment['feature'] = training_segment.apply(lambda x: get_segment_features(x), axis=1)
    training_segment = splitDataFrameList(training_segment, 'feature')
    training_segment['label'] = list(flatten(joblib.load('train_label.joblib')))
    training_segment = training_segment.drop(['segment','positions'], axis = 1)
    return training_segment

def get_test_data():
    test_segment = pd.read_csv('test_segment.txt', header=None, names = ['segment'])
    test_segment['feature'] = joblib.load('test_feature.joblib')
    test_segment['positions'] = test_segment.apply(lambda x: get_segment_positions(x), axis=1)
    test_segment['feature'] = test_segment.apply(lambda x: get_segment_features(x), axis=1)
    test_segment = splitDataFrameList(test_segment, 'feature')
    test_segment['ID'] = test_segment.index
    test_segment = test_segment.drop(['segment','positions'], axis = 1)
    return test_segment

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        ms = time.time()
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:                
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    if is_inception and phase == 'train':
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)           

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)            
            print('{} Loss: {:.4f}, {} Acc: {:.4f}'.format(phase, epoch_loss, phase, epoch_acc))
        
        time_taken = str(datetime.timedelta(seconds=time.time() - ms))
        print('time taken: {}'.format(time_taken))
            

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model, val_acc_history

class MyNet(Module):
    def __init__(self, batch_size, num_classes):
        super(MyNet, self).__init__()
        self.conv1 = Conv2d(in_channels=1, out_channels=batch_size, kernel_size=3, padding=1)
        self.conv1_bn = BatchNorm2d(num_features=batch_size)
        self.conv2 = Conv2d(in_channels=batch_size, out_channels=batch_size*2, kernel_size=3, padding=1)
        self.conv2_bn = BatchNorm2d(num_features=batch_size*2)
        self.conv3 = Conv2d(in_channels=batch_size*2, out_channels=batch_size*4, kernel_size=3, padding=1)
        self.conv3_bn = BatchNorm2d(num_features=batch_size*4)
        self.conv4 = Conv2d(in_channels=batch_size*4, out_channels=batch_size*8, kernel_size=3, padding=1)
        self.conv4_bn = BatchNorm2d(num_features=batch_size*8)
        self.pool = MaxPool2d(2, 2)
        self.drop1 = Dropout(0.5)
        self.linear1 = Linear(in_features=batch_size*8, out_features=batch_size*4)
        self.linear2 = Linear(in_features=batch_size*4, out_features=num_classes)
    
    def forward(self, x):
        x = self.pool(relu(self.conv1_bn(self.conv1(x))))
        #print("conv1", x.shape)
        x = self.pool(relu(self.conv2_bn(self.conv2(x))))
        #print("conv2",x.shape)
        x = self.pool(relu(self.conv3_bn(self.conv3(x))))
        #print("conv3",x.shape)
        x = self.pool(relu(self.conv4_bn(self.conv4(x))))
        #print("conv4",x.shape)
        x = self.drop1(x)
        x = x.view(-1, 512) 
        #print("x.view",x.shape)
        x = relu(self.linear1(x))
        #print("linear1",x.shape)
        x = self.drop1(x) 
        #x = softmax(self.linear2(x))
        x = self.linear2(x)
        return x

In [3]:
%%time
if reload_data:
    download_all_data_from_bucket()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.44 µs


In [5]:
%%time
train_df = get_train_data()
print(train_df.feature[0].shape)
train_df = splitDataFrameList(train_df, 'feature')
print(train_df.info())

torch.Size([260, 400])
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2855174 entries, 0 to 2855173
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   feature  object
 1   label    int64 
dtypes: int64(1), object(1)
memory usage: 43.6+ MB
None
CPU times: user 37 s, sys: 9.24 s, total: 46.3 s
Wall time: 46.2 s


In [6]:
%%time
train_sample_ratio = 1.0
train_data = train_df.sample(frac=train_sample_ratio)
train_data['feature'] = train_data.apply(lambda x : x["feature"].view(1, 20, 20), axis = 1)
train_data['label'] = train_data.apply(lambda x : x["label"] - 1, axis = 1)
print(train_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2855174 entries, 1615646 to 336068
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   feature  object
 1   label    int64 
dtypes: int64(1), object(1)
memory usage: 65.3+ MB
None
CPU times: user 1min 29s, sys: 108 ms, total: 1min 29s
Wall time: 1min 29s


In [7]:
%%time
num_classes = 47
batch_size = 64
num_epochs = 10
feature_extract = True

train_feature = torch.stack(train_data['feature'].tolist())
train_label = torch.tensor(train_data['label'].values.astype(np.long))

train_dataset = torch.utils.data.TensorDataset(train_feature, train_label)

train_size = int(0.98 * len(train_data))
val_size = int(0.01 * len(train_data))
test_size = len(train_data) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

dataloaders_dict = {'train':train_loader, 'val':test_loader}
print("len(train_loader.dataset) = ", len(train_loader.dataset))
print("len(val_loader.dataset) = ", len(val_loader.dataset))
print("len(test_loader.dataset) = ", len(test_loader.dataset))

len(train_loader.dataset) =  2798070
len(val_loader.dataset) =  28551
len(test_loader.dataset) =  28553
CPU times: user 8.28 s, sys: 3.33 s, total: 11.6 s
Wall time: 10.5 s


In [17]:
%%time
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#turn on cpu for troubleshooting as gpu doesnt throw proper error message, runing in cpu shows more specific error message
#device = torch.device("cpu") 
print(device)

model_ft = MyNet(batch_size, num_classes).double()
print(model_ft)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model_ft = DataParallel(model_ft)

model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

optimizer_ft = torch.optim.SGD(params_to_update, lr=0.001, momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=False)

cuda:0
MyNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3_bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4_bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.5, inplace=False)
  (linear1): Linear(in_features=512, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=47, bias=True)
)
Let's use 4 GPUs!
Params to learn:
	

In [19]:
%%time
model_name = "model_1.2_baseline.model" #renamed to model_1.2.model
torch.save(model_ft.state_dict(), model_name)
upload_files([model_name])

Upload from local model_1.2_baseline.model to data/breakfast-img-data/model_1.2_baseline.model
CPU times: user 40 ms, sys: 24 ms, total: 64 ms
Wall time: 768 ms


In [3]:
%%time
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
#turn on cpu for troubleshooting as gpu doesnt throw proper error message, runing in cpu shows more specific error message
#device = torch.device("cpu") 
print(device)

model_name = "model_1.2.model"
model = MyNet(64, 47).double().to(device)
model_dict = model.state_dict()
trained_model_dict = torch.load(model_name, map_location=device)
for k in model_dict:
    model_dict[k] = trained_model_dict["module." + k]
    
model.load_state_dict(model_dict)

cuda:0
CPU times: user 3.04 s, sys: 664 ms, total: 3.7 s
Wall time: 2.3 s


<All keys matched successfully>

In [8]:
%%time
correct = 0
total = 0
with torch.no_grad():
    for data in val_loader:
        inputs, labels = data[0], data[1]        
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of test data: {0}'.format(100 * correct / total))

Accuracy of test data: 90.95303141746349
CPU times: user 6.38 s, sys: 1.76 s, total: 8.13 s
Wall time: 7.87 s


In [9]:
%%time
test_df = get_test_data()
test_df = splitDataFrameList(test_df, 'feature')
test_df

CPU times: user 6.91 s, sys: 400 ms, total: 7.31 s
Wall time: 7.3 s


,feature,ID
0,"[tensor(-15.8486, dtype=torch.float64), tensor...",0
1,"[tensor(-15.7037, dtype=torch.float64), tensor...",0
2,"[tensor(-15.1998, dtype=torch.float64), tensor...",0
3,"[tensor(-15.2123, dtype=torch.float64), tensor...",0
4,"[tensor(-16.0185, dtype=torch.float64), tensor...",0
...,...,...
468908,"[tensor(-16.3263, dtype=torch.float64), tensor...",1283
468909,"[tensor(-14.9555, dtype=torch.float64), tensor...",1283
468910,"[tensor(-16.1000, dtype=torch.float64), tensor...",1283
468911,"[tensor(-15.5954, dtype=torch.float64), tensor...",1283


In [23]:
%%time
test_data = test_df.sample(frac = 1.0)
print(test_data.info())
test_data['feature'] = test_data.apply(lambda x : x["feature"].view(1, 20, 20), axis = 1)
test_feature = torch.stack(test_data['feature'].tolist())
test_dataset = torch.utils.data.TensorDataset(test_feature)
predict_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)
result = []
with torch.no_grad():
    for data in predict_loader:
        inputs = data[0]            
        inputs = inputs.to(device)    
        outputs = model(inputs)  
        prop = torch.nn.functional.softmax(outputs).cpu().tolist()
        result = np.append(result, prop)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 468913 entries, 42893 to 401625
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   feature  468913 non-null  object
 1   ID       468913 non-null  int64 
 2   label    468913 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 14.3+ MB
None


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


CPU times: user 3min 47s, sys: 2min 26s, total: 6min 14s
Wall time: 6min 14s


In [24]:
%%time
test_result = pd.DataFrame(result.reshape(-1,47))
test_result['ID'] = test_data['ID'].values
test_data_mean = test_result.groupby('ID')[list(range(47))].agg(['mean'])
test_data_mean.columns = list(range(47))
test_data_mean["Category"] = test_data_mean.idxmax(axis = 1)
test_data_mean['Category'] = test_data_mean.apply(lambda x : x["Category"] + 1, axis = 1).astype('int') 
test_data_mean["ID"] = test_data_mean.index
test_data_group = test_data_mean[['ID','Category']]
print(test_data_group.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1284 entries, 0 to 1283
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   ID        1284 non-null   int64
 1   Category  1284 non-null   int64
dtypes: int64(2)
memory usage: 30.1 KB
None
CPU times: user 528 ms, sys: 60 ms, total: 588 ms
Wall time: 583 ms


In [25]:
len(test_data_group.Category.unique())

35

In [26]:
submission_name = "submission_1.2_mean.csv"
test_data_group.to_csv(submission_name, index=False)
upload_files([submission_name])

Upload from local submission_1.2_mean.csv to data/breakfast-img-data/submission_1.2_mean.csv


In [30]:
%%time
test_df = get_test_data()
test_df = splitDataFrameList(test_df, 'feature')
test_df['feature'] = test_df.apply(lambda x : x["feature"].view(1, 20, 20), axis = 1)
test_feature = torch.stack(test_df['feature'].tolist())
test_dataset = torch.utils.data.TensorDataset(test_feature)
predict_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
result = []
with torch.no_grad():
    for data in predict_loader:
        inputs = data[0]
        inputs = inputs.to(device) 
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        result = np.append(result, predicted.cpu())

test_df['label'] = np.transpose(result).astype('int') 
test_df

CPU times: user 1min 52s, sys: 31.1 s, total: 2min 24s
Wall time: 2min 24s


,feature,ID,label
0,"[[[tensor(-15.8486, dtype=torch.float64), tens...",0,28
1,"[[[tensor(-15.7037, dtype=torch.float64), tens...",0,28
2,"[[[tensor(-15.1998, dtype=torch.float64), tens...",0,28
3,"[[[tensor(-15.2123, dtype=torch.float64), tens...",0,19
4,"[[[tensor(-16.0185, dtype=torch.float64), tens...",0,12
...,...,...,...
468908,"[[[tensor(-16.3263, dtype=torch.float64), tens...",1283,36
468909,"[[[tensor(-14.9555, dtype=torch.float64), tens...",1283,25
468910,"[[[tensor(-16.1000, dtype=torch.float64), tens...",1283,1
468911,"[[[tensor(-15.5954, dtype=torch.float64), tens...",1283,1


In [31]:
%%time
test_data_group = test_df.groupby('ID')['label'].apply(list).reset_index(name='labels')
test_data_group['Category'] = test_data_group.apply(lambda x : np.argmax(np.bincount(x.labels)) + 1, axis = 1)
test_data_group = test_data_group[['ID','Category']]
test_data_group

CPU times: user 200 ms, sys: 4 ms, total: 204 ms
Wall time: 202 ms


,ID,Category
0,0,14
1,1,32
2,2,2
3,3,11
4,4,29
...,...,...
1279,1279,44
1280,1280,29
1281,1281,15
1282,1282,2


In [32]:
test_data_group.Category.unique()

array([14, 32,  2, 11, 29, 15, 13, 21, 44, 12, 20, 25, 46, 39,  1, 37, 34,
       17, 27, 26,  5, 24,  6, 28, 19, 33, 45])

In [33]:
len(test_data_group.Category.unique())

27

In [34]:
submission_name = "submission_1.2_max.csv"
test_data_group.to_csv(submission_name, index=False)

In [35]:
upload_files([submission_name])

Upload from local submission_1.2_max.csv to data/breakfast-img-data/submission_1.2_max.csv


In [38]:
%%time
test_data = test_df.sample(frac = 1.0)
print(test_data.info())
test_data['feature'] = test_data.apply(lambda x : x["feature"].view(1, 20, 20), axis = 1)
test_feature = torch.stack(test_data['feature'].tolist())
test_dataset = torch.utils.data.TensorDataset(test_feature)
predict_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64)
result = []
with torch.no_grad():
    for data in predict_loader:
        inputs = data[0]            
        inputs = inputs.to(device)    
        outputs = model(inputs)  
        prop = outputs.cpu().tolist()
        result = np.append(result, prop)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 468913 entries, 262608 to 203380
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   feature  468913 non-null  object
 1   ID       468913 non-null  int64 
 2   label    468913 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 14.3+ MB
None
CPU times: user 3min 51s, sys: 2min 26s, total: 6min 17s
Wall time: 6min 17s


In [39]:
%%time
test_result = pd.DataFrame(result.reshape(-1,47))
test_result['ID'] = test_data['ID'].values
test_data_mean = test_result.groupby('ID')[list(range(47))].agg(['sum'])
test_data_mean.columns = list(range(47))
test_data_mean["Category"] = test_data_mean.idxmax(axis = 1)
test_data_mean['Category'] = test_data_mean.apply(lambda x : x["Category"] + 1, axis = 1).astype('int') 
test_data_mean["ID"] = test_data_mean.index
test_data_group = test_data_mean[['ID','Category']]
print(test_data_group.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1284 entries, 0 to 1283
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   ID        1284 non-null   int64
 1   Category  1284 non-null   int64
dtypes: int64(2)
memory usage: 30.1 KB
None
CPU times: user 512 ms, sys: 72 ms, total: 584 ms
Wall time: 583 ms


In [40]:
len(test_data_group.Category.unique())

38

In [41]:
submission_name = "submission_1.2_sum.csv"
test_data_group.to_csv(submission_name, index=False)
upload_files([submission_name])

Upload from local submission_1.2_sum.csv to data/breakfast-img-data/submission_1.2_sum.csv
